In [1]:
import numpy as np
from autode.evaluation import *
from autode.datagenerators import *
from autode.optimizers import *

# Run Example Evaluation

## Define $t$ and Generate Observations

In [2]:
gen = TankDataGenerator()
t = np.linspace(0,150,30)
obs = gen.generate_orig(t)
obs

array([[25.        ,  0.        ],
       [16.8834666 ,  6.88825478],
       [11.87653833,  9.30380954],
       [ 8.66155566,  9.55556714],
       [ 6.50769526,  8.839764  ],
       [ 5.00380371,  7.76290089],
       [ 3.91399052,  6.62128609],
       [ 3.09936019,  5.55003304],
       [ 2.4754182 ,  4.60211919],
       [ 1.98875132,  3.79004147],
       [ 1.60414837,  3.10752257],
       [ 1.29739749,  2.54062145],
       [ 1.05118717,  2.07325207],
       [ 0.85271822,  1.68978109],
       [ 0.69227001,  1.37612412],
       [ 0.56230764,  1.12009111],
       [ 0.45690282,  0.91137357],
       [ 0.37134187,  0.74137633],
       [ 0.30184941,  0.602996  ],
       [ 0.24538648,  0.49039514],
       [ 0.19949865,  0.39879419],
       [ 0.16219912,  0.32428905],
       [ 0.1318772 ,  0.2636957 ],
       [ 0.10722581,  0.21442006],
       [ 0.08718354,  0.17435011],
       [ 0.07088809,  0.14176706],
       [ 0.05763874,  0.11527258],
       [ 0.04686594,  0.09372924],
       [ 0.03810669,

## Create Evaluator Aware of the Data

In [3]:
evaluator = ODEINTEvaluator(t, obs)

## Evaluate Models

### Correct Model

In [39]:
print("Final Error:", evaluator.evaluate([get_func_from_str('a*x1 + b*x2', 2), get_func_from_str('c*x1 + x2', 2)], attempts_per_exponent=10))

Setting exponent to 0
Trying 0-th optimization for exp 0
Observed result <lmfit.minimizer.MinimizerResult object at 0x7fbde39d6750>
Found new best model with score 9.86276906952938e+74
Trying 1-th optimization for exp 0
Observed result <lmfit.minimizer.MinimizerResult object at 0x7fbde39b3190>
Trying 2-th optimization for exp 0
Observed result <lmfit.minimizer.MinimizerResult object at 0x7fbde3cb8e10>
Trying 3-th optimization for exp 0
Observed result <lmfit.minimizer.MinimizerResult object at 0x7fbde39b3150>
Trying 4-th optimization for exp 0
Observed result <lmfit.minimizer.MinimizerResult object at 0x7fbde3992410>
Trying 5-th optimization for exp 0
Observed result <lmfit.minimizer.MinimizerResult object at 0x7fbde3981950>
Trying 6-th optimization for exp 0
Observed result <lmfit.minimizer.MinimizerResult object at 0x7fbde3984810>
Trying 7-th optimization for exp 0


KeyboardInterrupt: 

In [ ]:
evaluator.params_for_last_model

### Wrong Models

#### Quadratic Model

In [5]:
%%time
print(evaluator.evaluate(lambda Y, t, a: Y**2 -4))

Setting exponent to 0
Trying 0-th optimization for exp 0


RuntimeError: The size of the array returned by func (1) does not match the size of y0 (2).

#### Sine Model

In [ ]:
%%time
print(evaluator.evaluate(lambda Y, t, a: np.sin(Y**2) * a))

## Find Best Model With Optimizer

In [ ]:
%%time
solver = NaiveOptimizer(2, 1, 2)
counter = 0
rows = []
for cand in solver.get_all_models():
    counter += 1
    print(f"Evaluating {cand}")
    score = evaluator.evaluate([get_func_from_str(cand[i], 2) for i in range(2)], attempts_per_exponent=10)
    rows.append([cand[0], cand[1], score])
counter

In [ ]:
import pandas as pd
df = pd.DataFrame(rows, columns=["$d x_1 / dt$", "$d x_2 / dt$", "error"])
df["error"] = np.round(df["error"], 2)
df

In [ ]:
df.sort_values("error")